# Clone DTRB repository from github 

In [2]:
!git clone https://github.com/clovaai/deep-text-recognition-benchmark.git

Cloning into 'deep-text-recognition-benchmark'...
remote: Enumerating objects: 499, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 499 (delta 0), reused 1 (delta 0), pack-reused 495
Receiving objects: 100% (499/499), 3.07 MiB | 7.05 MiB/s, done.
Resolving deltas: 100% (301/301), done.


In [3]:
%cd /content/deep-text-recognition-benchmark

/content/deep-text-recognition-benchmark


## Extract dataset that uploaded in google drive

In [ ]:
!pip install -q patool

In [ ]:
import patoolib
patoolib.extract_archive("/content/drive/MyDrive/dataset/Persian_pelate/validation.rar", outdir="/content/drive/MyDrive/dataset/Persian_pelate")

## Download pretrained model from google drive 

In [5]:
!gdown 1b59rXuGGmKne1AuHnkgDzoYgKeETNMv9

Downloading...
From: https://drive.google.com/uc?id=1b59rXuGGmKne1AuHnkgDzoYgKeETNMv9
To: /content/deep-text-recognition-benchmark/TPS-ResNet-BiLSTM-Attn.pth
100% 199M/199M [00:02<00:00, 92.3MB/s]


In [6]:
!gdown 1ubkg7E2vGEOqS4K_quwf9Vl-i8IVpklM

Downloading...
From: https://drive.google.com/uc?id=1ubkg7E2vGEOqS4K_quwf9Vl-i8IVpklM
To: /content/deep-text-recognition-benchmark/plate_img-train.zip
100% 196M/196M [00:06<00:00, 28.6MB/s]


In [7]:
!gdown 1AL5Zsg2hDqcwF8ZmR0MJTbjgXIoE5W-I

Downloading...
From: https://drive.google.com/uc?id=1AL5Zsg2hDqcwF8ZmR0MJTbjgXIoE5W-I
To: /content/deep-text-recognition-benchmark/plate_img-validation.zip
100% 27.2M/27.2M [00:00<00:00, 50.6MB/s]


In [10]:
!pip install -q lmdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 6.3 MB/s eta 0:00:00


In [ ]:
!python3 demo.py \
--Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn \
--image_folder my_image/ --batch_max_length 70 \
--saved_model TPS-ResNet-BiLSTM-Attn.pth

model input parameters 32 100 20 1 512 256 38 70 TPS ResNet BiLSTM Attn
loading pretrained model from TPS-ResNet-BiLSTM-Attn.pth
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
--------------------------------------------------------------------------------
image_path               	predicted_labels         	confidence score
--------------------------------------------------------------------------------
my_image/english1.jpg    	ppz65pwo                 	0.0168
my_image/english2.jpg    	intercomentalize         	0.0000
my_image/farsi1.jpg      	intarvity            

In [9]:
!pip install -q fire

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


## Convert persian plate dataset to lmdb file

In [ ]:
!python3 create_lmdb_dataset.py --inputPath /content/drive/MyDrive/dataset/Persian_pelate --gtFile /content/drive/MyDrive/dataset/Persian_pelate/gt_train.txt --outputPath /content/drive/MyDrive/dataset/Persian_pelate/lmdb_files

In [ ]:
!python3 create_lmdb_dataset.py --inputPath /content/drive/MyDrive/dataset/Persian_pelate --gtFile /content/drive/MyDrive/dataset/Persian_pelate/gt_validation.txt --outputPath /content/drive/MyDrive/dataset/Persian_pelate/lmdb_files_validation

In [13]:
!python3 create_lmdb_dataset.py --inputPath /content/drive/MyDrive/dataset/Persian_pelate --gtFile /content/drive/MyDrive/dataset/Persian_pelate/gt_test.txt --outputPath /content/drive/MyDrive/dataset/Persian_pelate/lmdb_files_test

Written 1000 / 5559
Written 2000 / 5559
Written 3000 / 5559
Written 4000 / 5559
Written 5000 / 5559
Created dataset with 5559 samples


## Train

In [ ]:
!python3 train.py \
--train_data /content/drive/MyDrive/dataset/Persian_pelate/lmdb_files_train --valid_data /content/drive/MyDrive/dataset/Persian_pelate/lmdb_files_validation \
--select_data / --batch_ratio 1  --num_iter 16000 --exp_name /content/drive/MyDrive/dataset/best_model_dtrb\
--Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn

Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: /content/drive/MyDrive/dataset/Persian_pelate/lmdb_files_train
opt.select_data: ['/']
opt.batch_ratio: ['1']
--------------------------------------------------------------------------------
dataset_root:    /content/drive/MyDrive/dataset/Persian_pelate/lmdb_files_train	 dataset: /
sub-directory:	/.	 num samples: 19358
num total samples of /: 19358 x 1.0 (total_data_usage_ratio) = 19358
num samples of / per batch: 192 x 1.0 (batch_ratio) = 192
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation 

# Test

In [15]:
!python3 test.py \
--eval_data /content/drive/MyDrive/dataset/Persian_pelate/lmdb_files_test \
--saved_model /content/drive/MyDrive/dataset/best_model_dtrb/best_accuracy.pth \
--Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn

model input parameters 32 100 20 1 512 256 38 25 TPS ResNet BiLSTM Attn
loading pretrained model from /content/drive/MyDrive/dataset/best_model_dtrb/best_accuracy.pth
dataset_root:    /content/drive/MyDrive/dataset/Persian_pelate/lmdb_files_test	 dataset: /
sub-directory:	/.	 num samples: 5552
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
80.439


# Read Text on 5 test persian plate

In [11]:
!python3 demo.py \
--Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn \
--image_folder /content/drive/MyDrive/dataset/5_plate_test_image --batch_max_length 8 \
--saved_model /content/drive/MyDrive/dataset/best_model_dtrb/best_accuracy.pth

model input parameters 32 100 20 1 512 256 38 8 TPS ResNet BiLSTM Attn
loading pretrained model from /content/drive/MyDrive/dataset/best_model_dtrb/best_accuracy.pth
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
--------------------------------------------------------------------------------
image_path               	predicted_labels         	confidence score
--------------------------------------------------------------------------------
/content/drive/MyDrive/dataset/5_plate_test_image/00022.jpg	28i21255                 	0.9970
/content/drive/MyDrive/dataset/5_pl